In [ ]:
!pip install sentence-transformers faiss-cpu transformers datasets tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 82.9 MB/s eta 0:00:00


In [1]:
import json
import os
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

/home/naif/miniconda3/envs/legal-assistant/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def extract_articles_v2(dataset):
    articles = []
    for category, subcats in dataset.items():
        for subcat, systems in subcats.items():
            for system_name, system_data in systems.items():
                brief = system_data.get("brief", "")
                metadata = system_data.get("metadata", {})
                parts = system_data.get("parts", {})
                for part_name, part_articles in parts.items():
                    for article in part_articles:
                        articles.append({
                            "category": category,
                            "sub_category": subcat,
                            "system": system_name,
                            "part": part_name,
                            "brief": brief,
                            "metadata": metadata,
                            "id": article.get("id"),
                            "title": article.get("Article_Title"),
                            "status": article.get("status"),
                            "text": article.get("Article_Text")
                        })
    return articles


In [3]:
with open("data/saudi_laws_scraped.json", "r", encoding="utf-8") as f:
    data = json.load(f)

articles = extract_articles_v2(data)
print(f"✅ Total Articles Extracted: {len(articles)}")
print(json.dumps(articles[604], indent=2, ensure_ascii=False))

✅ Total Articles Extracted: 16371
{
  "category": "أنظمة عادية",
  "sub_category": "الأمن الداخلي والأحوال المدنية والأنظمة الجنائية",
  "system": "نظام مكافحة غسل الأموال",
  "part": "main",
  "brief": "يتضمن النظام:\r\nالمقصود بالعبارات والألفاظ الواردة بالنظام. الأفعال التي يعد مرتكبها مرتكبًا جريمة غسل الأموال – ما يجب على المؤسسات المالية وغير المالية اتخاذه من إجراءات حيال مرتكب جريمة غسل الأموال – البرامج التي تضعها المؤسسات المالية وغير المالية لمكافحة عمليات غسل الأموال – وحدة مكافحة غسل الأموال – عقوبة مرتكب جريمة غسل الأموال.",
  "metadata": {
    "الاسم": "نظام مكافحة غسل الأموال",
    "تاريخ الإصدار": "1433/05/11 هـ  الموافق : 03/04/2012 مـ",
    "تاريخ النشر": "1433/08/02  هـ الموافق : 22/06/2012 مـ",
    "الحالة": "لاغي",
    "أدوات إصدار النظام": [
      {
        "text": "مرسوم ملكي رقم م / 31 بتاريخ 11 / 5 / 1433",
        "url": "https://laws.boe.gov.sa/BoeLaws/Laws/Viewer/9ec732e6-9bbf-4fda-8a61-a9ae00c3c014?lawId=4a8842df-9cd1-4ee7-bf97-a9a700f180d4"
      },
     

In [4]:
[article['text'] for i, article in  enumerate(articles) if article['id'] == 2]

['يكون\nعلم الدولة\nكما يلي :\nأ  - لونه أخضر.\nب - عرضه يساوي ثلثي طوله.\nج - تتوسطه كلمة : (لا إله إلا الله محمد رسول الله) تحتها سيف مسلول، ولا ينكس العلم أبدا.\nويبين  النظام  الأحكام المتعلقة به.']

In [5]:
def build_corpus(articles):
    corpus = []
    for art in articles:
        title = art.get("title", "").strip()
        brief = art.get("brief", "").strip()
        text = art.get("text", "").strip()
        
        # Format metadata as "key: value" pairs
        meta = art.get("metadata", {})
        meta_str = " ".join(f"{k}: {v}" for k, v in meta.items() if v)

        # Combine elements with clean formatting
        parts = [
            f"Law Title: {title}" if title else "",
            f"Law Brief: {brief}" if brief else "",
            f"Law Text: {text}" if text else "",
            f"Law Metadata: {meta_str}" if meta_str else "",
        ]

        # Filter out empty parts and join with double newlines for clarity
        entry = "\n\n".join(filter(None, parts)).strip()
        corpus.append(entry)

    return corpus


corpus = build_corpus(articles)
print(f"✅ Corpus built with {len(corpus)} documents")
for i in range(2):
    print(f"\n--- Example {i+1} ---\n{corpus[i][:400]}...")

✅ Corpus built with 16371 documents

--- Example 1 ---
Law Title: المادة الأولى

Law Brief: يتضمن العناوين التالية: المبادئ العامة، نظام الحكم، مقومات المجتمع السعودي، المبادئ الاقتصادية، الحقوق والواجبات، سلطات الدولة، الشئون المالية، أحكام عامة.

Law Text: المملكة العربية السعودية، دولة عربية إسلامية، ذات
سيادة تامة
، دينها
الإسلام
، ودستورها
كتاب الله تعالى
وسنة رسوله صلى الله عليه وسلم. ولغتها هي اللغة العربية، وعاصمتها مدينة الرياض.

Law Metadata...

--- Example 2 ---
Law Title: المادة الثانية

Law Brief: يتضمن العناوين التالية: المبادئ العامة، نظام الحكم، مقومات المجتمع السعودي، المبادئ الاقتصادية، الحقوق والواجبات، سلطات الدولة، الشئون المالية، أحكام عامة.

Law Text: عيدا الدولة، هما عيدا الفطر والأضحى، وتقويمها، هو
التقويم الهجري.

Law Metadata: الاسم: النظام الأساسي للحكم تاريخ الإصدار: 1412/08/27 هـ  الموافق : 01/03/1992 مـ تاريخ النشر: 1412/09/02  هـ المو...


In [6]:
import torch

embed_model = SentenceTransformer("BAAI/bge-m3", device="cuda" if torch.cuda.is_available() else "cpu")

In [7]:
batch_size = 128
num_items = len(corpus)
dim = embed_model.get_sentence_embedding_dimension()
embeddings = np.zeros((num_items, dim), dtype=np.float32)

for start in tqdm(range(0, num_items, batch_size)):
    end = start + batch_size
    batch = corpus[start:end]
    embeddings[start:end] = embed_model.encode(batch, show_progress_bar=False, convert_to_numpy=True, normalize_embeddings=True)



100%|██████████| 128/128 [08:26<00:00,  3.96s/it]


In [10]:
print(f"✅ Embeddings shape: {embeddings.shape}")

✅ Embeddings shape: (16371, 1024)


In [11]:
dim = embeddings.shape[1]
base_index = faiss.IndexFlatIP(dim)
index = faiss.IndexIDMap(base_index)
ids = np.arange(embeddings.shape[0])
index.add_with_ids(embeddings, ids)
faiss.write_index(index, "m3_legal_faiss_brief.index")

In [10]:
index = faiss.read_index("data/m3_legal_faiss.index")
print("--- FAISS INDEX DEBUG ---")
print(f"Loaded index type: {type(index)}")
print(f"Total vectors: {index.ntotal}")
# If it's an IndexIDMap, it will have a sub-index called 'index'
print(f"Is IndexIDMap? {hasattr(index, 'index')}")
if hasattr(index, 'index'):
    print(f"Sub-index type: {type(index.index)}")
print("--------------------------")

--- FAISS INDEX DEBUG ---
Loaded index type: <class 'faiss.swigfaiss_avx2.IndexIDMap'>
Total vectors: 16371
Is IndexIDMap? True
Sub-index type: <class 'faiss.swigfaiss_avx2.Index'>
--------------------------


In [17]:

filtered_indices = np.array([0, 1,2, 5 ], dtype=np.int64)
selector = faiss.IDSelectorArray(filtered_indices)

def retrieve(query, top_k=5):
    q_emb = embed_model.encode([query], convert_to_numpy=True, normalize_embeddings=True)
    D, I = index.search(q_emb, top_k, params=faiss.SearchParameters(sel=selector)
)
    results = [(i, float(D[0][j])) for j, i in enumerate(I[0])]
    return results

query = "ما هو شعار الدولة السعودية؟"
results = retrieve(query)

for i, (idx, score) in enumerate(results, 1):
    print(f"\n🔹 Result ID {idx} (score={score:.3f})\n{corpus[idx][:400]}...")


🔹 Result ID 0 (score=0.578)
المادة الأولى - المملكة العربية السعودية، دولة عربية إسلامية، ذات
سيادة تامة
، دينها
الإسلام
، ودستورها
كتاب الله تعالى
وسنة رسوله صلى الله عليه وسلم. ولغتها هي اللغة العربية، وعاصمتها مدينة الرياض. الاسم: النظام الأساسي للحكم تاريخ الإصدار: 1412/08/27 هـ  الموافق : 01/03/1992 مـ تاريخ النشر: 1412/09/02  هـ الموافق : 06/03/1992 مـ الحالة: ساري أدوات إصدار النظام: [{'text': 'أمر ملكي رقم أ/90 بتاري...

🔹 Result ID 2 (score=0.562)
المادة الثالثة - يكون
علم الدولة
كما يلي :
أ  - لونه أخضر.
ب - عرضه يساوي ثلثي طوله.
ج - تتوسطه كلمة : (لا إله إلا الله محمد رسول الله) تحتها سيف مسلول، ولا ينكس العلم أبدا.
ويبين  النظام  الأحكام المتعلقة به. الاسم: النظام الأساسي للحكم تاريخ الإصدار: 1412/08/27 هـ  الموافق : 01/03/1992 مـ تاريخ النشر: 1412/09/02  هـ الموافق : 06/03/1992 مـ الحالة: ساري أدوات إصدار النظام: [{'text': 'أمر ملكي رقم...

🔹 Result ID 1 (score=0.400)
المادة الثانية - عيدا الدولة، هما عيدا الفطر والأضحى، وتقويمها، هو
التقويم الهجري. الاسم: النظام الأساسي 